In [2]:
def get_mouth_height(top_lip, bottom_lip):
    sum=0
    for i in [8,9,10]:
        # distance between two near points up and down
        distance = math.sqrt( (top_lip[i][0] - bottom_lip[18-i][0])**2 + 
                              (top_lip[i][1] - bottom_lip[18-i][1])**2   )
        sum += distance
    return sum / 3

In [4]:
import math
from mouth_open_algorithm import get_lip_height, get_mouth_height, check_mouth_open

# obama open mouth
#top_lip = [(181, 359), (192, 339), (211, 332), (225, 336), (243, 333), (271, 342), (291, 364), (282, 363), (242, 346), (225, 347), (211, 345), (188, 358)]
#bottom_lip = [(291, 364), (270, 389), (243, 401), (223, 403), (207, 399), (190, 383), (181, 359), (188, 358), (210, 377), (225, 381), (243, 380), (282, 363)]

# close mouth
#top_lip = [(151, 127), (157, 126), (163, 126), (168, 127), (172, 127), (178, 127), (185, 129), (182, 129), (172, 130), (167, 130), (163, 129), (153, 127)]
#bottom_lip = [(185, 129), (177, 133), (171, 135), (166, 135), (161, 134), (156, 132), (151, 127), (153, 127), (162, 129), (167, 130), (171, 130), (182, 129)]

print('top_lip height: %.2f' % get_lip_height(top_lip))
print('bottom_lip height: %.2f' % get_lip_height(bottom_lip))
print('mouth height: %.2f' % get_mouth_height(top_lip,bottom_lip))
print('Is mouth open:', check_mouth_open(top_lip,bottom_lip) )

top_lip height: 3.05
bottom_lip height: 5.07
mouth height: 0.67
Is mouth open: False


In [7]:
from PIL import Image, ImageDraw
import face_recognition
import cv2

#webcam = cv2.VideoCapture(1)
#success, images = webcam.read()
images = 

# Load the jpg file into a numpy array
image = face_recognition.load_image_file(images)

# Find all facial features in all the faces in the image
face_landmarks_list = face_recognition.face_landmarks(image)

print("I found {} face(s) in this photograph.".format(len(face_landmarks_list)))

pil_image = Image.fromarray(image)
d = ImageDraw.Draw(pil_image)

for face_landmarks in face_landmarks_list:
    # Print the location of each facial feature in this image
    facial_features = [
        'chin',
        'left_eyebrow',
        'right_eyebrow',
        'nose_bridge',
        'nose_tip',
        'left_eye',
        'right_eye',
        'top_lip',
        'bottom_lip'
    ]

    for facial_feature in facial_features:
        print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))

    # Let's trace out each facial feature in the image with a line!
    for facial_feature in facial_features:
        d.line(face_landmarks[facial_feature], width=5)

# Display drawed image
cv2.imshow("Face Detection", image)
cv2.waitKey(0)
#pil_image.save('test.png')

AttributeError: 'numpy.ndarray' object has no attribute 'read'

In [2]:
import face_recognition
import cv2
from mouth_open_algorithm import get_lip_height, get_mouth_height
from datetime import datetime

def is_mouth_open(face_landmarks):
    top_lip = face_landmarks['top_lip']
    bottom_lip = face_landmarks['bottom_lip']

    top_lip_height = get_lip_height(top_lip)
    bottom_lip_height = get_lip_height(bottom_lip)
    mouth_height = get_mouth_height(top_lip, bottom_lip)
    
    # if mouth is open more than lip height * ratio, return true.
    ratio = 0.5
    print('top_lip_height: %.2f, bottom_lip_height: %.2f, mouth_height: %.2f, min*ratio: %.2f' 
          % (top_lip_height,bottom_lip_height,mouth_height, min(top_lip_height, bottom_lip_height) * ratio))
          
    if mouth_height > min(top_lip_height, bottom_lip_height) * ratio:
        return True
    else:
        return False


# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(1)

# Define the codec and create VideoWriter object to save video to local
fourcc = cv2.VideoWriter_fourcc(*'XVID') # codec
# cv2.VideoWriter( filename, fourcc, fps, frameSize )
out = cv2.VideoWriter('output.avi', fourcc, 7, (640, 480)) 

# Load a sample picture and learn how to recognize it.
pk_image = face_recognition.load_image_file("PK2.jpg") # replace peter.jpg with your own image !!
pk_face_encoding = face_recognition.face_encodings(pk_image)[0]

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    # Find all the faces and face enqcodings in the frame of video
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    face_landmarks_list = face_recognition.face_landmarks(frame)

    # Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding, face_landmarks in zip(face_locations, face_encodings, face_landmarks_list):

        #  See if the face is a match for the known face(s)
        match = face_recognition.compare_faces([pk_face_encoding], face_encoding)

        name = "Unknown"
        if match[0]:
            name = "PK"

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom), (right, bottom + 35), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom + 25), font, 1.0, (255, 255, 255), 1)


        # Display text for mouth open / close
        ret_mouth_open = is_mouth_open(face_landmarks)
        if ret_mouth_open is True:
            text = 'Mouth is open'
        else:
            text = 'Open your mouth'
        cv2.putText(frame, text, (left, top - 50), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)


    # Display the resulting image
    cv2.imshow('Video', frame)
    out.write(frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

top_lip_height: 10.23, bottom_lip_height: 15.83, mouth_height: 21.50, min*ratio: 5.11
top_lip_height: 13.77, bottom_lip_height: 15.90, mouth_height: 4.45, min*ratio: 6.89
top_lip_height: 9.04, bottom_lip_height: 13.03, mouth_height: 6.49, min*ratio: 4.52
top_lip_height: 11.36, bottom_lip_height: 12.74, mouth_height: 2.85, min*ratio: 5.68
top_lip_height: 10.35, bottom_lip_height: 11.03, mouth_height: 3.33, min*ratio: 5.17
top_lip_height: 11.00, bottom_lip_height: 12.04, mouth_height: 3.33, min*ratio: 5.50
top_lip_height: 10.00, bottom_lip_height: 12.03, mouth_height: 5.03, min*ratio: 5.00
top_lip_height: 12.01, bottom_lip_height: 12.71, mouth_height: 3.39, min*ratio: 6.01
top_lip_height: 9.35, bottom_lip_height: 11.03, mouth_height: 2.00, min*ratio: 4.67
top_lip_height: 11.02, bottom_lip_height: 13.01, mouth_height: 4.33, min*ratio: 5.51
top_lip_height: 10.33, bottom_lip_height: 13.03, mouth_height: 6.00, min*ratio: 5.17
top_lip_height: 11.33, bottom_lip_height: 14.01, mouth_height: 5.0

KeyboardInterrupt: 

: 

In [1]:
from PIL import Image, ImageDraw
import face_recognition

image_file = 'PK2.jpg' 

# Load the jpg file into a numpy array
image = face_recognition.load_image_file(image_file)

# Find all facial features in all the faces in the image
face_landmarks_list = face_recognition.face_landmarks(image)

print("I found {} face(s) in this photograph.".format(len(face_landmarks_list)))

pil_image = Image.fromarray(image)
d = ImageDraw.Draw(pil_image)

for face_landmarks in face_landmarks_list:
    # Print the location of each facial feature in this image
    facial_features = [
        'chin',
        'left_eyebrow',
        'right_eyebrow',
        'nose_bridge',
        'nose_tip',
        'left_eye',
        'right_eye',
        'top_lip',
        'bottom_lip'
    ]

    for facial_feature in facial_features:
        print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))

    # Let's trace out each facial feature in the image with a line!
    for facial_feature in facial_features:
        d.line(face_landmarks[facial_feature], width=5)

# Display drawed image
pil_image.show()

I found 1 face(s) in this photograph.
The chin in this face has the following points: [(1524, 1994), (1523, 2124), (1538, 2263), (1574, 2405), (1631, 2539), (1714, 2659), (1807, 2760), (1909, 2829), (2047, 2837), (2205, 2810), (2358, 2757), (2500, 2674), (2620, 2558), (2694, 2406), (2720, 2233), (2716, 2054), (2707, 1878)]
The left_eyebrow in this face has the following points: [(1534, 1864), (1567, 1783), (1644, 1741), (1741, 1723), (1837, 1735)]
The right_eyebrow in this face has the following points: [(1987, 1699), (2103, 1643), (2231, 1624), (2355, 1652), (2459, 1719)]
The nose_bridge in this face has the following points: [(1921, 1872), (1920, 1957), (1915, 2040), (1910, 2129)]
The nose_tip in this face has the following points: [(1831, 2267), (1888, 2276), (1949, 2277), (2017, 2262), (2081, 2243)]
The left_eye in this face has the following points: [(1632, 1962), (1682, 1917), (1749, 1905), (1820, 1941), (1759, 1965), (1690, 1976)]
The right_eye in this face has the following poi